In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm, trange
tqdm.pandas()

import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from numba import jit, njit

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Import data

In [2]:
df = pd.read_csv('../data/wikihowAll.csv', delimiter=',', nrows=10000)

In [3]:
df.shape

(10000, 3)

Датасет дуже великий і його обробка займає дуже багато часу, тому візьму перші 1000 рядків. В майбутньому збільшу кількість.

In [4]:
df = df[:1000]

In [5]:
df.shape

(1000, 3)

In [6]:
df.head()

,headline,title,text
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,How to Become an Art Investor,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ..."


In [7]:
print(df['headline'][0])


Keep related supplies in the same area.,
Make an effort to clean a dedicated workspace after every session.,
Place loose supplies in large, clearly visible containers.,
Use clotheslines and clips to hang sketches, photos, and reference material.,
Use every inch of the room for storage, especially vertical space.,
Use chalkboard paint to make space for drafting ideas right on the walls.,
Purchase a label maker to make your organization strategy semi-permanent.,
Make a habit of throwing out old, excess, or useless stuff each month.


In [8]:
print(df['text'][0])

 If you're a photographer, keep all the necessary lens, cords, and batteries in the same quadrant of your home or studio. Paints should be kept with brushes, cleaner, and canvas, print supplies should be by the ink, etc. Make broader groups and areas for your supplies to make finding them easier, limiting your search to a much smaller area. Some ideas include:


Essential supplies area -- the things you use every day.
Inspiration and reference area.
Dedicated work area .
Infrequent or secondary supplies area, tucked out of the way.;
, This doesn't mean cleaning the entire studio, it just means keeping the area immediately around the desk, easel, pottery wheel, etc. clean each night. Discard trash or unnecessary materials and wipe down dirty surfaces. Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.


Even if the rest of your studio is a bit disorganized, an organized workspace will help 

Можна вважати вміст колонки 'headline' підсумованим текстом колонки 'text'. В майбутньому можна використати для оцінки побудованої моделі. Колонку 'title' я не використовуватиму, то її видалю.

In [9]:
df.drop('title', inplace=True, axis=1)

In [10]:
df.shape

(1000, 2)

In [11]:
df['text'].isnull().sum()

4

Чотири рядки мають значення Nan тому їх теж треба видалити

In [12]:
df.dropna(subset = ["text"], inplace=True)

In [13]:
df['text'].isnull().sum()

0

In [14]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def clean_text(texts):
    corpus = []
    
    for t in tqdm(texts):
        text = re.sub(r"(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", t)
        text = re.sub(r"\d+", "", text)
        text = text.lower()
        text = text.split()
        t = []
        for word in text:
            if word not in stop_words:
                t.append(word)
        t = ' '.join(t)
        corpus.append(t)
        
    return corpus

In [16]:
sentences = df['text'].progress_apply(lambda x: x).values

In [17]:
df['text'] = clean_text(sentences)

In [18]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [19]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [20]:
def word_lemmatizer(texts):
    corpus = []
    lemmatizer = WordNetLemmatizer()
    for text in tqdm(texts):
        t = []
        for i in word_tokenize(text):            
            lem_text = lemmatizer.lemmatize(i, get_wordnet_pos(i))
            t.append(lem_text)
        t = ' '.join(t)
        corpus.append(t)
    return corpus

In [21]:
sentences = df['text'].progress_apply(lambda x: x).values

In [22]:
df['text'] = word_lemmatizer(sentences)

In [23]:
df.head()

,headline,text
0,"\nKeep related supplies in the same area.,\nMa...",youre photographer keep necessary lens cord ba...
1,\nCreate a sketch in the NeoPopRealist manner ...,see image draw develops stepbystep however imp...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",possible become vfx artist without college deg...
3,\nStart with some experience or interest in ar...,best art investor research piece art buy someo...
4,"\nKeep your reference materials, sketches, art...",start planning project work youll likely gathe...


In [24]:
# sentences = df['headline'].progress_apply(lambda x: x).values
# df['headline'] = clean_text(sentences)

In [25]:
# sentences = df['headline'].progress_apply(lambda x: x).values
# df['headline'] = word_lemmatizer(sentences)

In [26]:
# df.head()

In [27]:
# df['headline'][0]

In [29]:
df.head()

,headline,text
0,"\nKeep related supplies in the same area.,\nMa...",youre photographer keep necessary lens cord ba...
1,\nCreate a sketch in the NeoPopRealist manner ...,see image draw develops stepbystep however imp...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",possible become vfx artist without college deg...
3,\nStart with some experience or interest in ar...,best art investor research piece art buy someo...
4,"\nKeep your reference materials, sketches, art...",start planning project work youll likely gathe...


In [33]:
df.to_csv(r'../data/wikihowAll_cleaned.csv', index=False)

In [35]:
dfq = pd.read_csv('../data/wikihowAll_cleaned.csv', delimiter=',')

In [36]:
dfq

,headline,text
0,"\nKeep related supplies in the same area.,\nMa...",youre photographer keep necessary lens cord ba...
1,\nCreate a sketch in the NeoPopRealist manner ...,see image draw develops stepbystep however imp...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",possible become vfx artist without college deg...
3,\nStart with some experience or interest in ar...,best art investor research piece art buy someo...
4,"\nKeep your reference materials, sketches, art...",start planning project work youll likely gathe...
...,...,...
991,\nApproach your hamster cautiously with both h...,make sure palm remember quietly
992,"\nProvide enough room for your guinea pig.,\nE...",guinea pig enough room move unhappy stress mig...
993,"\nUse a cage that is a tank.,\nClean the cage ...",prepare hamster baby use cage tanklike ie like...
994,"\nMonitor your guinea pig during floor time.,\...",take piggie cage play explore leave alone alwa...
